In [37]:
# Necessary Import Statements
import pandas as pd
import numpy as np

# Imports the datasets
data = pd.read_csv('Exoplanet_Data.csv', low_memory=False)
phl = pd.read_csv('PHL_Dataset.csv', low_memory=False)


In [38]:
# Removes any planets that have an NA eccentricity value
data = data[data['pl_orbeccen'].notna()]
data = data[data['pl_rade'].notna()]

# Resets the data row index
data = data.reset_index (drop = True)

In [39]:
# Makes an empty m_type dataframe with same column headers
k_type = pd.DataFrame(columns=list(data.columns))

# Makes an empty m_type_habitable dataframe to store planets that result out of this code
k_type_habitable = pd.DataFrame(columns=list(data.columns))

# Fills the m_type dataframe with M-type host stars' planets
k_type = data.loc[(data['st_spectype'].str.startswith('K', na=False))]

# Resets the m_type row index
k_type = k_type.reset_index (drop = True)

In [40]:
# Takes out planets that have no planet radius and eccentricity
phl = phl[phl['pl_rade'].notna()]
phl = phl[phl['pl_orbeccen'].notna()]

# Resets index
phl = phl.reset_index (drop = True)

In [41]:
# Makes a new empty dataframe for the m-type planets in the PHL dataset
phl_k_type = pd.DataFrame(columns=list(phl.columns))

In [42]:
# Fills the empty dataframe with m-type planets from the PHL dataset
phl_k_type = phl.loc[(phl['st_spectype'].str.startswith('K', na=False))]

# Resets index
phl_k_type = phl_k_type.reset_index (drop = True)

In [43]:
def check_match(planet, control_planet):
    if pd.isna(planet['pl_dens']):
        if (not pd.isna(planet['pl_radj'])) & (not pd.isna(planet['pl_massj'])):
            planet['pl_dens'] = (planet['pl_massj'] * (1.898 * (10 ** 27) * 1000)) \
            / (4 * np.pi * ((planet['pl_radj'] * 43441 * 160934) ** 3) / 3)
        else:
            return
        
    if pd.isna(planet['pl_orbsmax']) and (pd.isna(planet['pl_ratdor']) or pd.isna(planet['st_rad'])):
        return

    # Calculates orbital period in days for main planet
    if pd.isna(planet['pl_orbper']):
        if pd.isna(planet['pl_ratdor']):
            planet['pl_orbper'] = np.sqrt(planet['pl_orbsmax']**3) * 365
        elif not pd.isna(planet['st_rad']): 
            planet['pl_orbper'] = np.sqrt((planet['pl_ratdor']*planet['st_rad'])**3) * 365
        if pd.isna(planet['pl_orbper']):
            return
    
    if pd.isna(control_planet['pl_orbsmax']) and (pd.isna(control_planet['pl_ratdor']) or pd.isna(control_planet['st_rad'])):
        return

    # Calculates orbital period in days for phl planet
    if pd.isna(control_planet['pl_orbper']):
        if pd.isna(control_planet['pl_ratdor']):
            control_planet['pl_orbper'] = np.sqrt(control_planet['pl_orbsmax']**3) * 365
        elif not pd.isna(control_planet['st_rad']): 
            control_planet['pl_orbper'] = np.sqrt((control_planet['pl_ratdor']*control_planet['st_rad'])**3) * 365
        if pd.isna(control_planet['pl_orbper']):
            return
    
    if pd.isna(planet['pl_orbeccen']):
        return
    
    

    if (float(planet['pl_dens']) >= 4) & (float(planet['pl_dens']) <= 7):
        if (planet['pl_controv_flag'] == '0'):
            if ('Transit' in planet['discoverymethod']):
                if (planet['pl_orbeccen'] <= 0.2):
                    if (np.abs(float(planet['pl_orbper']) - float(control_planet['pl_orbper'])) <= 50):
                        global k_type_habitable
                        k_type_habitable = k_type_habitable.append(planet, ignore_index = True)


In [44]:
for i in range (len(k_type['pl_rade'])): 
    current_dist = np.abs(phl_k_type['pl_rade'][0]-k_type['pl_rade'][i])
    if current_dist < 20:
        planet = k_type.loc[i].copy()
        control_planet = phl_k_type.loc[0].copy()
        check_match(planet, control_planet)

In [45]:
k_type_habitable

,rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,default_flag,sy_snum,...,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,Unnamed: 287,Unnamed: 288,Unnamed: 289,Unnamed: 290,Unnamed: 291,Unnamed: 292
0,383,GJ 143 b,GJ 143,b,HD 21749,HIP 16069,TIC 279741379,Gaia DR2 4673947174316727040,1,1,...,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2467,K2-110 b,K2-110,b,NaN,NaN,TIC 287333762,Gaia DR2 3613175223837135616,0,1,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
